<img width="8%" alt="Naas.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Naas.png" style="border-radius: 15%">

# Naas - Send notification

**Tags:** #naas #emailbuilder #snippet #operations

**Author:** [Florent Ravenel](https://www.linkedin.com/in/ACoAABCNSioBW3YZHc2lBHVG0E_TXYWitQkmwog/)

**Description:** This notebook send a notification with your latest posts performance.

## Input

### Import libraries

In [ ]:
from naas_drivers import emailbuilder, gsheet
import naas
import os
import naas_data_product
from datetime import date, datetime, timedelta
from naas_drivers import naasauth

### Setup variables
**Inputs**
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `input_dir`: Input directory to retrieve file from.
- `datalake_dir`: Datalake directory to retrieve plugin json file.

**Outputs**
- `email_to`: This variable is used for storing a list of email addresses that will receive the notification email. In this case, the list contains the email address of the script user, which is retrieved using the naasauth.connect().user.me().get("username") method.
- `subject`: This variable is used for storing the subject of the notification email. In this case, the subject is "🖼️ Content Engine: LinkedIn profile update".

In [ ]:
# Inputs
input_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "sales-engine", date.today().isoformat())
input_file = "sales"
input_image = "sales_trend.png"
input_html = "sales_trend.html"
datalake_dir = naas_data_product.OUTPUTS_PATH
spreadsheet_url = naas.secret.get("ABI_SPREADSHEET") or "YOUR_GOOGLE_SPREADSHEET_URL"
sheet_name_input = "SALES"

# Outputs
email_to = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entity"), "emails") or []
entity_name = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entity"), "entity_name") or []
email_subject = f"⚡️ Sales Assistant Report - {entity_name} ({datetime.now().strftime('%Y-%m-%d %H:%M:%S')})"
output_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "content-engine", date.today().isoformat())
csv_output = os.path.join(output_dir, f"{datetime.now().strftime('%Y%m%d%H%M%S')}_sales-assistant-report_{entity_name.lower().replace(' ', '_')}.csv")

## Model

### Get assets

In [ ]:
image_url = get_image_asset(input_dir, input_image)
print("Image URL:", image_url)

image_html = get_image_asset(input_dir, input_html)
print("Image HTML:", image_html)

plugin_url = naas.asset.find(os.path.join(datalake_dir, "plugins", "🚀_growth_assistant.json"))
print("Plugin URL:", plugin_url)

### Get data from Google Sheets spreadsheet

In [ ]:
df_input = pload(input_dir, input_file)    
if df_input is None:
    df_input = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name_input)

# Filter on this week and last week
tw = date.today().strftime("W%W-%Y")
lw = (date.today() - timedelta(days=date.today().weekday() + 7)).strftime("W%W-%Y")
df_input = df_input[df_input["SCENARIO"].isin([tw, lw])]

# Display result
df_input.to_csv(csv_output, index=False)
print("Rows:", len(df_input))
df_input.head(1)

### Prep companies for email

In [ ]:
staff_maping = {
    "Solopreneur": 1,
    "Micro Team": 2,
    "Small Company": 3,
    "Medium Company": 4,
    "Large Company": 5,
    "Enterprise Level": 6,
    "Major Corporation": 7,
    "Global Corporation": 8,
    "Mega Corporation": 9,
    "NA": 0,
}

to_group = [
    "COMPANY_NAME",
    "COMPANY_URL",
    "STAFF_RANGE_NAME",
    "STAFF_COUNT",
    "COUNTRY",
]
to_agg = {
    "INTERACTION_SCORE": "sum"
}
df_companies = df_input.groupby(to_group, as_index=False).agg(to_agg)
df_companies = df_companies[(df_companies["COMPANY_NAME"] != "TBD") & (df_companies["STAFF_RANGE_NAME"] != "NA")]
df_companies.STAFF_COUNT = df_companies.STAFF_COUNT.astype(str).str.replace("TBD", "0").astype(int)
df_companies["COMPANY_SIZE_ORDER"] = df_companies["STAFF_RANGE_NAME"].map(staff_maping)
df_companies = df_companies.sort_values(by=["COMPANY_SIZE_ORDER", "STAFF_COUNT"], ascending=[False, False]).reset_index(drop=True)

companies = []
for x in df_companies["STAFF_RANGE_NAME"].unique():
    company = f"{x}:"
#     print(company)
    tmp_df = df_companies[df_companies["STAFF_RANGE_NAME"] == x].reset_index(drop=True)
    for row in tmp_df.itertuples():
        index = row.Index
        name = row.COMPANY_NAME
        url = row.COMPANY_URL
        company = f"{company}<ul><a href='{url}'>{name}</a></ul>"
    companies.append(company)

### Prep messagings option for email

In [ ]:
messagings = []
for row in df_input.itertuples():
    index = row.Index
    fullname = row.FULLNAME
    profile_url = row.PROFILE_URL
    occupation = row.OCCUPATION
    company = row.COMPANY_NAME
    messagings_options = row.MESSAGING_OPTIONS
    if messagings_options != "TBD":
        option1 = messagings_options.split('1.')[1].split('2.')[0]
        option2 = messagings_options.split('2.')[1].split('3.')[0]
        option3 = messagings_options.split('3.')[1]
        messagings.append(f"<a href='{profile_url}'>{fullname}</a> - '{occupation}' at {company}:<br><span style='font-size: 14px;'><ul>1.{option1}</ul><ul>2.{option2}</ul><ul>3.{option3}</ul></span>") 
    else:
        messagings.append(f"<a href='{profile_url}'>{fullname}</a> - '{occupation}' at {company}")

### Create Email brief

In [ ]:
def email_brief(
    companies,
    messagings,
    plugin_url,
    image_url,
    image_html,
):
    content = {
        'heading': "⚡️ Sales Assistant",
        "txt_intro": (
            f"Hi {entity_name.split(' ')[0]},<br><br>" "I hope this email finds you well. Here is the latest update on your sales performance."
        ),
        "title_1": emailbuilder.text(
            "Overview", font_size="25px", text_align="center", bold=True
        ),
        "image_1": emailbuilder.image(image_url, image_html),
        "title_2": emailbuilder.text(
            "Performance", font_size="25px", text_align="center", bold=True
        ),
        "text_1": emailbuilder.text(
            "Companies breadown:",
            underline=True,
        ),
        "list_1": emailbuilder.list(companies),
        "text_2": emailbuilder.text(
            "New sales qualified leads:",
            underline=True,
        ),
        "list_2": emailbuilder.list(messagings),
        "button_1": emailbuilder.button(
            link=(f"https://naas.ai/chat/use?plugin_url={plugin_url}"),
            text="Start Chatting",
            background_color="#181a1c",
        ),
        "footer_cs": emailbuilder.footer_company(naas=True),
    }

    email_content = emailbuilder.generate(display="iframe", **content)
    return email_content


email_content = email_brief(
    companies,
    messagings,
    plugin_url,
    image_url,
    image_html
)

## Output

### Send the email

In [ ]:
naas.notification.send(
    email_to=email_to,
    subject=email_subject,
    html=email_content,
    files=[csv_output],
)